In [ ]:
import sys
import os
import SimpleITK as sitk

current_directory = os.getcwd()
path = os.path.dirname(current_directory)
sys.path.append(path)
from sitkTools import show_image

%matplotlib widget
from ipywidgets import interact, interactive, widgets
from matplotlib.patches import Rectangle, Circle, Arrow

# Set paths and files

In [ ]:
import glob
import os

# import numpy as np
import matplotlib.pyplot as plt
from typing import Optional, Any, Mapping, Hashable

import monai
from monai.transforms import (
    Compose,
    EnsureChannelFirst,
    EnsureChannelFirstd,
    Orientationd
)
from sitkMonaiIO import LoadSitkImage, LoadSitkImaged, PushSitkImage, PushSitkImaged

# Image files
file_list = sorted(glob.glob(os.path.join(current_directory, '*.nii.gz')))
selected_file = file_list[0]

sitk_loaded = sitk.ReadImage(selected_file, sitk.sitkFloat32)
print('SimpleITK object sizes')
print(sitk_loaded.GetSize())
show_image(sitk_loaded, 'Sitk from file')

# Create dictionary for images
sitk_dict = {'image': sitk_loaded}

# Use LoadSitkImage to obtain metatensors from sitk
loadSitk = Compose([LoadSitkImaged(keys=['image'], image_only=True),
                   EnsureChannelFirstd(keys=['image']),
                   Orientationd(keys=['image'], axcodes='PIL')
                   ])
metatensor = loadSitk(sitk_dict)
print('Metatensor sizes')
print(metatensor['image'].shape)
    
# Use PushSitkImage to obtain sitk from metatensors
pushSitk = Compose([PushSitkImaged(keys=['image'], resample=True, print_log=False)])
sitk_output = pushSitk(metatensor)
print(sitk_output['image'].GetSize())
show_image(sitk_output['image'], 'Sitk from metatensor')